In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='h_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for col in pr_list:
                pr_arr_row.append(row[col])
                #print(f'{col}: {row[col]}', end='; ')
            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float32)
    y = np.asarray(y_arr, dtype=np.float32)
    return X, y

#### Сумма, средний, высший, низший уровни

In [3]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность S

In [4]:
def get_s(h_max, h_forecast=None):
    if h_forecast is None:
        # Среднеквадратическая погрешность климатическая
        return np.std(h_max, ddof=1)
    else:
        # Среднеквадратическая погрешность прогноза
        n = h_max.shape[0]
        sqr_diff = np.sum((h_max - h_forecast) ** 2) / (n - 1)
        std = sqr_diff ** 0.5
        return std    

#### Среднеквадратическое отклонение sigma

In [5]:
def get_sigma(h_max):
    return get_s(h_max)

#### Допустимая погрешность прогноза delta_dop

In [6]:
def get_delta_dop(sigma):
    return 0.674 * sigma

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [7]:
def get_criterion(s, sigma):
    #print(s / sigma)
    return s / sigma

#### Климатическая обеспеченность Pk

In [8]:
def get_pk(h_max, delta_dop):
    avg_level = np.mean(h_max)
    diff = np.abs(h_max - avg_level)
    trusted_values = diff[diff <= delta_dop]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Обеспеченность метода (оправдываемость) Pm

In [9]:
def get_pm(h_max, h_forecast, delta_dop):
    diff = np.abs(h_max - h_forecast) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    #print(m, n)
    return m / n * 100.00

#### Корреляционное отношение

In [10]:
def get_correlation_ratio(criterion):
    c_1 = (1 - criterion ** 2)
    ro = c_1 ** 0.5 if c_1 > 0 else 0
    return ro

#### Вероятная ошибка прогноза S'

In [11]:
def get_forecast_error(s):
    return 0.674 * s

#### Ошибки климатического/природного прогноза для каждого года delta50

In [1]:
def get_delta50(h_max, delta_dop, h_forecast=None):
    if h_forecast is None:
        # delta50 климатическая
        avg = np.mean(h_max)
        return np.abs(h_max - avg) / delta_dop
    else:
        # delta50 прогнозов
        avg = np.mean(h_max)
        return np.abs(h_max - h_forecast) / delta_dop
  

#### Функция записи в csv файл

In [13]:
import csv
def write_dataset_csv(dataset, filename, fieldnames):
    with open(f'results/{filename}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


#### Функция разделения набора данных на тренировочный и тестовый

In [14]:
def train_test_split(X, y, n_test):
    X_train = X[:-n_test]
    y_train = y[:-n_test]
    X_test = X[-n_test:]
    y_test = y[-n_test:]
    return X_train, y_train, X_test, y_test

#### Функция формирования проверочных прогнозов

In [ ]:
def verify_forecast(h_forecast):

    ds_dir = 'data' # В константы
    
    datasets = [ 
        ('Неман-Белица', 'Неман')
    ] # В константы

    for ds in datasets:
        dataset_result = []
        year, h_max = get_river_dataset(f'{ds_dir}/{ds[0]}-2022.csv', pr_list=['year'], y_name='h_max')

        # Hсредний
        h_avg = np.mean(h_max)

        # H - Hсредний
        diff_fact = h_max - h_avg

        # (H - Hсредний) в квадрате
        diff_fact_sqr = diff_fact ** 2

        # Погрешность климатических прогнозов в долях от допустимой погрешности
        delta_dop = get_delta_dop(get_sigma(h_max))
        error_climate = get_delta50(h_max, delta_dop)

        # H - Hпрогнозный
        diff_forecast = h_max - h_forecast

        # (H - Hпрогнозный) в квадрате
        diff_forecast_sqr = f_forecast ** 2       

        # Погрешность проверочных прогнозов в долях от допустимой погрешности
        error_forecast = get_delta50(h_max, delta_dop, h_forecast)

        # Конкатенация массивов
        att_tuple = (year, h_max, diff_fact, diff_fact_sqr, error_climate, h_forecast, diff_forecast, diff_forecast_sqr, error_forecast)
        arr = np.column_stack(att_tuple)


        
        
        

#### Функция обучения и оценки моделей

In [15]:
def compare_models(validation=False, n_test=9, norms=False):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet, ElasticNetCV
    from sklearn.linear_model import Lars, LarsCV
    from sklearn.linear_model import LassoLars

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
    
    
    ds_dir = 'data' # В константы
    
    names = [
        'LinearRegression',
        'Ridge',
        'Lasso',
        'ElasticNet',
        'ElasticNetCV',
        'LassoLars',
        'Lars1',
        'Lars2',
        'Lars3',
        'Lars4',
        'Lars5',
        'Lars6',
        'Lars7',
        'Lars8',
        'LarsCV',
        
        #'GaussianProcessRegressor',
        
    ]

    regressors = [
        LinearRegression(),
        Ridge(alpha=10),
        Lasso(alpha=100.0),
        ElasticNet(alpha=2.0, l1_ratio=0.1),
        ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.01, n_alphas=1000),
        LassoLars(alpha=1.0),
        Lars(n_nonzero_coefs=1),
        Lars(n_nonzero_coefs=2),
        Lars(n_nonzero_coefs=3),
        Lars(n_nonzero_coefs=4),
        Lars(n_nonzero_coefs=5),
        Lars(n_nonzero_coefs=6),
        Lars(n_nonzero_coefs=7),
        Lars(n_nonzero_coefs=8),
        LarsCV(max_iter=5000, max_n_alphas=10000, cv=3),

        #GaussianProcessRegressor(kernel=RBF(length_scale=1.1) + WhiteKernel() + DotProduct(), random_state=0)
    ]

    datasets = [
        ('Неман-Белица', 'Неман')
    ]

    predictors_lists = {
        'Неман': ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    }

    norms = {
        
    }

    fieldnames = ['Predictors', 'Equations', 'Method', 'Criterion', 'Correlation', 'Pm']
    
    # Итерация по датасетам
    for ds in datasets:
        dataset_result = []
        X, y = get_river_dataset(f'{ds_dir}/{ds[0]}-2022.csv', pr_list=predictors_lists[ds[1]])

        if validation:
            X_train, y_train, X_test, y_test = train_test_split(X, y, n_test)
        else:
            X_train = X[:]
            y_train = y[:]
            X_test = X_train
            y_test = y_train
            
        # Итерация по моделям регрессии
        for name, regr in zip(names, regressors):
            model_row = dict()
                
            regr = regr.fit(X_train, y_train)
            y_predicted = regr.predict(X_test)
            
            # Коэффициенты уравнения (если есть)
            try:
                predictors_coef = {f: c for f, c in zip(predictors_lists[ds[1]], regr.coef_) if c != 0.0}
                predictors = ", ".join(predictors_coef.keys())
                equation = ' '.join(str(round(c, 2))+'*'+f for f, c in predictors_coef.items())
                equation = equation.replace(" -", "-")
                equation = equation.replace(" ", " + ")
                equation = equation.replace("-", " - ")
    
                model_row['Predictors'] = predictors
                model_row['Equations'] = equation
            except Exception:
                model_row['Predictors'] = ""
                model_row['Equations'] = ""

            # Название метода
            model_row['Method'] = name

            # Расчет показателей качества по методике
            
            sigma = get_sigma(y)
            delta_dop = get_delta_dop(sigma)
            s_forecast = get_s(y_test, y_predicted)
            
            
            # Критерий эффективности метода прогнозирования климатический S/sigma
            criterion_forecast = get_criterion(s_forecast, sigma)
            model_row['Criterion'] = round(criterion_forecast, 2)
            
            # Корреляционное отношение ro
            correlation_forecast = get_correlation_ratio(criterion_forecast)
            model_row['Correlation'] = round(correlation_forecast, 2)
            
            # Обеспеченность метода (оправдываемость) Pm
            pm = get_pm(y_test, y_predicted, delta_dop)
            model_row['Pm'] = round(pm, 2)

            # Model
            model_row['Model'] = regr

            dataset_result.append(model_row)

            # Сортировка результатов по каждому датасету
            dataset_result.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))

            # Запись в .csv файл
            write_dataset_csv(dataset_result, ds[0], fieldnames)

    return dataset_result

In [16]:
result = compare_models(validation=True, n_test=9)
print(*result, sep='\n')

{'Predictors': 's_2802, h, x, x2', 'Equations': '0.91*s_2802 + 0.2*h + 0.39*x + 0.71*x2', 'Method': 'Lars4', 'Criterion': 0.75, 'Correlation': 0.66, 'Pm': 77.78, 'Model': Lars(n_nonzero_coefs=4)}
{'Predictors': 's_2802, h, x, x2, x3', 'Equations': '0.91*s_2802 + 0.21*h + 0.39*x + 0.72*x2 + 0.0*x3', 'Method': 'Lars5', 'Criterion': 0.75, 'Correlation': 0.66, 'Pm': 77.78, 'Model': Lars(n_nonzero_coefs=5)}
{'Predictors': 's_2802, s_max, h, x, x2, x3', 'Equations': '0.91*s_2802 + 0.0*s_max + 0.21*h + 0.39*x + 0.72*x2 + 0.01*x3', 'Method': 'Lars6', 'Criterion': 0.75, 'Correlation': 0.66, 'Pm': 77.78, 'Model': Lars(n_nonzero_coefs=6)}
{'Predictors': 's_2802, x, x2', 'Equations': '0.84*s_2802 + 0.3*x + 0.25*x2', 'Method': 'Lars3', 'Criterion': 0.76, 'Correlation': 0.65, 'Pm': 55.56, 'Model': Lars(n_nonzero_coefs=3)}
{'Predictors': 's_2802, x', 'Equations': '0.77*s_2802 + 0.26*x', 'Method': 'Lars2', 'Criterion': 0.78, 'Correlation': 0.63, 'Pm': 44.44, 'Model': Lars(n_nonzero_coefs=2)}
{'Predict

In [17]:
new_prediction = []

### Формирование набора данных

In [18]:
#pr_list = ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs']

In [19]:
pr_list = ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'xs']

In [20]:
X, y = get_river_dataset('data/Неман-Белица-2022.csv', pr_list=pr_list)

### Нормализация данных

In [21]:
# Стандартизация
scaler = preprocessing.StandardScaler().fit(X)
X_train_st = scaler.transform(X)


### Модели регрессии

#### Линейная регрессия

In [22]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr = regr.fit(X, y)
y_predicted = regr.predict(X)
print(y_predicted)

[286.93356 332.20343 450.00058 305.71146 272.2185  328.75146 274.95145
 247.11786 309.54523 357.50894 308.29926 302.80093 198.77216 247.6426
 225.88866 259.27112 359.91058 209.0416  392.24146 367.39508 240.4675
 265.16626 312.1084  309.3382  343.23138 307.57138 255.45236 257.1446
 365.3783  281.29486 210.21957 393.6142  155.48703 238.59187 272.66876
 261.3518  209.70857]


### Вычисление статистики

In [23]:
# Среднеквадратическое отклонение sigma
sigma = get_sigma('Неман-Белица')
print(f'Среднеквадратическое отклонение sigma = {sigma}')

# Допустимая погрешность прогноза delta_dop
delta_dop = get_delta_dop(sigma)
print(f'Допустимая погрешность прогноза sigma_dop = {delta_dop}')

F:\ruslan\projects\pkogo\env_ogo\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U12'), dtype('<U12')) -> None

#### Климат

In [ ]:
# Статистика максимального уровня
sum_level = get_sum(y)
avg_level = get_avg(y)
max_level = get_max(y)
min_level = get_min(y)
print(f'Сумма {sum_level}, Средний {avg_level}, Высший {max_level}, Низший {min_level}')

# Среднеквадратическая погрешность климатическая S
s_climate = get_s(y)
print(f'Среднеквадратическая погрешность климатическая: {s_climate}')

# Критерий эффективности метода прогнозирования климатический S/sigma
criterion_climate = get_criterion(s_climate, sigma)
print(f'Критерий эффективности метода прогнозирования климатический S/sigma = {criterion_climate}')

# Климатическая обеспеченность Pk
pk = get_pk(y, delta_dop)
print(f'Климатическая обеспеченность Pk, % = {pk}')

# Корреляционное отношение ro
ro_climate = get_correlation_ratio(criterion_climate)
print(f'Корреляционное отношение климатическое ro = {ro_climate}')

# Вероятная погрешность прогноза delta50
delta50_climate = get_delta50(s_climate)
print(f'Вероятная погрешность климатическая delta50 = {delta50_climate}')

#### Прогноз

In [ ]:
# Коэффициенты регрессии
print(regr.intercept_, regr.coef_)

# Среднеквадратическая погрешность климатическая S
s_forecast = get_s(y, y_predicted)
print(f'Среднеквадратическая погрешность прогноза: {s_forecast}')

# Критерий эффективности метода прогнозирования климатический S/sigma
criterion_forecast = get_criterion(s_forecast, sigma)
print(f'Критерий эффективности метода прогнозирования S/sigma = {criterion_forecast}')

# Обеспеченность метода (оправдываемость) Pm
pm = get_pm(y, y_predicted, delta_dop)
print(f'Обеспеченность метода (оправдываемость) Pm, % = {pm}')

# Корреляционное отношение ro
ro_forecast = get_correlation_ratio(criterion_forecast)
print(f'Корреляционное отношение ro = {ro_forecast}')

# Вероятная погрешность прогноза delta50
delta50_forecast = get_delta50(s_forecast)
print(f'Вероятная погрешность прогноза delta50 = {delta50_forecast}')

### Обучающий набор

In [ ]:
test_qnt = 5

In [ ]:
X_train = X[:-test_qnt]
X_test = X[-test_qnt:]

In [ ]:
y_train = y[:-test_qnt]
y_test = y[-test_qnt:]

## Этап нормализации